# Supabase Storage and DuckDB: A Powerful Combination for Data Handling

⚡️ + 🦆 = 💚

This demo showcases the seamless integration between Supabase Storage for data storage and DuckDB for efficient querying. Explore how to leverage these powerful tools together to streamline your data workflows.

## Table of Contents

0. [Setup and Installation](#Setup-and-Installation)
1. [Query Postgres Data and Export to Supabase Storage](#Query-Postgres-Data-and-Export-to-Supabase-Storage)
2. [Query Supabase Storage Directly with DuckDB](#Query-Supabase-Storage-Directly-with-DuckDB)
3. [Visualizing Results](#Visualizing-Results)

## Setup and Installation

### Install Packages
comment out the below block if you need to install the packages into your environment.

In [ ]:
# !pip install -r https://raw.githubusercontent.com/TylerHillery/supabase-storage-duckdb-demo/main/requirements.txt?token=GHSAT0AAAAAACPZEHJ7KIXE6CPID5MDMUE6ZQSAINQ 

Then we will import the necessary packages for the demo

In [47]:
import os
from datetime import datetime

import altair as alt
import duckdb

The below two lines take environment variables from a .env file. If you are not using a .env file feel free to skip it. 

In [ ]:
from dotenv import load_dotenv

load_dotenv()

We will define some variables used for configuration management.

In [ ]:
# Postgres credentials
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DATABASE = os.getenv("POSTGRES_DATABASE")
POSTGRES_USERNAME = os.getenv("POSTGRES_USERNAME")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")

# Supabase storage credentials
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_REGION = os.getenv("AWS_REGION")
BUCKET_URL = os.getenv("BUCKET_URL")
ENDPOINT_URL = os.getenv("ENDPOINT_URL")

DuckDB has native support for the [S3 API](https://duckdb.org/docs/extensions/httpfs/s3api.html) and we can use the DuckDB [Secrets Manager](https://duckdb.org/docs/configuration/secrets_manager) to store our credentials 

In [4]:
duckdb.sql(f"""
DROP SECRET IF EXISTS supabase_storage;
CREATE SECRET supabase_storage (
    TYPE S3,
    KEY_ID '{AWS_ACCESS_KEY_ID}',
    SECRET '{AWS_SECRET_ACCESS_KEY}', 
    ENDPOINT '{ENDPOINT_URL}', 
    REGION '{AWS_REGION}',
    URL_STYLE 'path'
)
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

Load helpful DuckDB Postgres extension.

In [5]:
duckdb.sql("INSTALL postgres")

duckdb.sql(f"""
ATTACH 'dbname={POSTGRES_DATABASE} user={POSTGRES_USERNAME} host={POSTGRES_HOST} password={POSTGRES_PASSWORD} port={POSTGRES_PORT}' AS postgres_db (TYPE POSTGRES, READ_ONLY)
""")

## Query Postgres Data and Export to Supabase Storage

Lets first query data in our Postgres database and look at some results

In [25]:
select_pg_customers = """
SELECT
    id                              AS user_id,
    first_name                      AS user_first_name,
    last_name                       AS user_last_name,
    first_name || ' ' || last_name  AS user_full_name,
    CURRENT_DATE                    as loaded_at_date,
    CURRENT_TIMESTAMP               AS loaded_at_ts_utc
FROM 
    postgres_db.customers 
"""
duckdb.sql(select_pg_customers).df().head()

,user_id,user_first_name,user_last_name,user_full_name,loaded_at_date,loaded_at_ts_utc
0,1,Michael,P.,Michael P.,2024-04-07,2024-04-06 17:35:32.538000-07:00
1,2,Shawn,M.,Shawn M.,2024-04-07,2024-04-06 17:35:32.538000-07:00
2,3,Kathleen,P.,Kathleen P.,2024-04-07,2024-04-06 17:35:32.538000-07:00
3,4,Jimmy,C.,Jimmy C.,2024-04-07,2024-04-06 17:35:32.538000-07:00
4,5,Katherine,R.,Katherine R.,2024-04-07,2024-04-06 17:35:32.538000-07:00


Now lets export those query results to our Supabase Storage bucket as a parquet file.

In [26]:
duckdb.sql(f"""
COPY ({select_pg_customers}) 
TO '{BUCKET_URL}/customers/{datetime.now().strftime("%Y-%m-%d")}.parquet'
""")

You can do CSV file as well

In [20]:
duckdb.sql(f"""
COPY ({select_pg_customers}) 
TO '{BUCKET_URL}/customers/{datetime.now().strftime("%Y-%m-%d")}.csv'
""")

Lets do the same for the rest of the tables

In [21]:
select_pg_orders = """
SELECT
    id                  AS order_id,
    user_id             AS user_id,
    order_date          AS order_date,
    status              AS order_status,
    CURRENT_DATE        AS loaded_at_date,
    CURRENT_TIMESTAMP   AS loaded_at_ts_utc
FROM 
    postgres_db.orders
"""
duckdb.sql(select_pg_orders).df().head()

,order_id,user_id,order_date,order_status,loaded_at_date,loaded_at_ts_utc
0,1,1,2018-01-01,returned,2024-04-07,2024-04-06 17:18:09.396000-07:00
1,2,3,2018-01-02,completed,2024-04-07,2024-04-06 17:18:09.396000-07:00
2,3,94,2018-01-04,completed,2024-04-07,2024-04-06 17:18:09.396000-07:00
3,4,50,2018-01-05,completed,2024-04-07,2024-04-06 17:18:09.396000-07:00
4,5,64,2018-01-05,completed,2024-04-07,2024-04-06 17:18:09.396000-07:00


In [22]:
duckdb.sql(f"""
COPY ({select_pg_orders}) 
TO '{BUCKET_URL}/orders/{datetime.now().strftime("%Y-%m-%d")}.parquet'
""")

In [24]:
select_pg_payments = """
SELECT
    id                  AS payment_id,
    order_id            AS order_id,
    payment_method      AS payment_method,
    amount              AS order_amount_usd,
    CURRENT_DATE        AS loaded_at_date,
    CURRENT_TIMESTAMP   AS loaded_at_ts_utc
FROM 
    postgres_db.payments
"""
duckdb.sql(select_pg_payments).df().head()

,payment_id,order_id,payment_method,order_amount_usd,loaded_at_date,loaded_at_ts_utc
0,1,1,credit_card,1000.0,2024-04-07,2024-04-06 17:25:39.773000-07:00
1,2,2,credit_card,2000.0,2024-04-07,2024-04-06 17:25:39.773000-07:00
2,3,3,coupon,100.0,2024-04-07,2024-04-06 17:25:39.773000-07:00
3,4,4,coupon,2500.0,2024-04-07,2024-04-06 17:25:39.773000-07:00
4,5,5,bank_transfer,1700.0,2024-04-07,2024-04-06 17:25:39.773000-07:00


In [43]:
duckdb.sql(f"""
COPY ({select_pg_payments}) 
TO '{BUCKET_URL}/payments/{datetime.now().strftime("%Y-%m-%d")}.parquet'
""")

## Query Supabase Storage Directly with DuckDB

Now the tables are loaded lets query the files from Supabase Storage. We can even do file globbing patterns and return the filename.

In [38]:
select_from_bucket = f"""
SELECT count(*) as record_count
FROM read_parquet('{BUCKET_URL}/orders/*', filename = true)
"""
duckdb.sql(select_from_bucket).show()

┌──────────────┐
│ record_count │
│    int64     │
├──────────────┤
│           99 │
└──────────────┘



In [46]:
join_tables = f"""
SELECT
    orders.order_date,
    orders.order_id,
    customers.user_full_name,
    orders.order_status,
    payments.payment_method,
    payments.order_amount_usd,
    customers.user_id,
    payments.payment_id
FROM
    read_parquet('{BUCKET_URL}/orders/*.parquet') AS orders 
    LEFT JOIN read_parquet('{BUCKET_URL}/customers/*.parquet') AS customers
        ON orders.user_id = customers.user_id
    LEFT JOIN read_parquet('{BUCKET_URL}/payments/*.parquet') AS payments 
        ON orders.order_id = payments.order_id
"""
orders_df = duckdb.sql(join_tables).df()

orders_df.head()

,order_date,order_id,user_full_name,order_status,payment_method,order_amount_usd,user_id,payment_id
0,2018-01-01,1,Michael P.,returned,credit_card,1000.0,1,1
1,2018-01-02,2,Kathleen P.,completed,credit_card,2000.0,3,2
2,2018-01-04,3,Gregory H.,completed,coupon,100.0,94,3
3,2018-01-05,5,David C.,completed,bank_transfer,1700.0,64,5
4,2018-01-07,6,Rose M.,completed,credit_card,600.0,54,6


## Visualizing Results

DuckDB can return results in variety of formats such as:
- `.fetchall()` returns a list of tuples
- `.df()` returns a Pandas DataFrame
- `.pl()` returns a Polars DataFrame
- `.arrow()` return an Arrow Table 
- `fetchnumpy()` returns NumPy Arrays

This works nicely with several data viz libraries e.g. Matplotlib, Seaborn, Bokeh. I will be using Altair which is based Vega-Lite

In [61]:
monthly_sales = """
SELECT 
    date_trunc('month', order_date) as order_month,
    sum(order_amount_usd) as sales 
FROM 
    orders_df
WHERE 
    order_status != 'returned'
GROUP BY ALL
ORDER BY 1 
"""

chart = alt.Chart(duckdb.sql(monthly_sales).df()).mark_line().encode(
    x="order_month",
    y=alt.Y("sales", axis=alt.Axis(format='$,.2f')),
    tooltip=["order_month", "sales"]
).properties(
    width=750,
    height=400
).interactive()
chart

alt.Chart(...)

In [64]:
monthly_sales_by_order_status = """
SELECT 
    date_trunc('month', order_date)::string as order_month,
    order_status,
    sum(order_amount_usd) as sales 
FROM 
    orders_df
GROUP BY ALL
ORDER BY 1 
"""

chart = alt.Chart(duckdb.sql(monthly_sales_by_order_status).df()).mark_bar().encode(
    x="order_month",
    y=alt.Y("sales", axis=alt.Axis(format='$,.2f')),
    tooltip=["order_month", "sales", "order_status"],
    color="order_status",
).properties(
    width=750,
    height=400
).interactive()
chart

alt.Chart(...)